In [110]:
import numpy as np

## Logistic Regression Implementation

In [111]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

In [112]:
def calculate_gradient(theta, X, y):
    m = y.size      # This is the number of instances
    return (X.T @ (sigmoid(X @ theta) - y)) / m

In [113]:
def gradient_descent(X, y, alpha=0.1, num_iterations=100, tol=1e-7):
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]

    theta = np.zeros(X_bias.shape[1])

    for i in range(num_iterations):
        gradient = calculate_gradient(theta, X_bias, y)
        theta -= alpha * gradient

        if np.linalg.norm(gradient) < tol:
            break

    return theta

In [114]:
def predict_proba(X, theta):
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]

    return sigmoid(X_bias @ theta)


In [115]:
def predict(X, theta, threshold=0.5):
    return (predict_proba(X, theta) >= threshold).astype(int)

In [116]:
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [117]:
iris_data = load_iris()
X, y = iris_data.data, iris_data.target

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [119]:
# Training one classifier for each class using the One Vs Rest method (class 0 or non-class 0 and so on)
all_thetas = []
num_classes = len(np.unique(y))     # There are 3 unique classes in the Iris dataset

print(f"Training {num_classes} separate binary classifiers")

for i in range(num_classes):
    # Create a temporary target vector for this specific class
    y_train_specific = (y_train == i).astype(int)

    print(f"    Training classifier for class {i} ({iris_data.target_names[i]}) vs the rest")

    # Using the gradient descent function to train the binary classifier
    theta_hat = gradient_descent(X_train_scaled, y_train_specific, alpha=0.1, num_iterations=1000)

    # Store the trained parameters
    all_thetas.append(theta_hat)

print("Training complete")

Training 3 separate binary classifiers
    Training classifier for class 0 (setosa) vs the rest
    Training classifier for class 1 (versicolor) vs the rest
    Training classifier for class 2 (virginica) vs the rest
Training complete


In [120]:
def predict_ovr(X, thetas):
    probabilities = np.array([predict_proba(X, theta) for theta in thetas]).T

    return np.argmax(probabilities, axis=1)

In [121]:
# Predictions on the test set
y_pred = predict_ovr(X_test_scaled, all_thetas)

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"\nModel Accuracy: {accuracy:.4f}")

# Classification report for a more detailed look
print("\nClassification report")
print(classification_report(y_test, y_pred, target_names=iris_data.target_names))


Model Accuracy: 0.9667

Classification report
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.89      0.94         9
   virginica       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.96      0.97        30
weighted avg       0.97      0.97      0.97        30



## K Nearest Neighbors (KNN) Implementation

In [122]:
from collections import Counter

In [123]:
def calc_euclidean_dist(p, q):
    # This is a np.array of every value in p - every value in q
    #np.array(p) - np.array(q)

    return np.sqrt(np.sum((np.array(p) - np.array(q)) ** 2))

In [124]:
class KNeighborsClassifier:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_trian, y_train):
        self.X_train = X_trian
        self.y_train = y_train

    def predict_single(self, x_test_point):
        '''Helper function to predict the label of one single data point'''

        # Calculate the distance to all training points
        distances = [calc_euclidean_dist(x_test_point, x_train_point) for x_train_point in self.X_train]

        # Get the indices of the k nearest neighbors
        k_neigbors_indices = np.argsort(distances)[:self.k]

        # Get the labels of the neighbors
        k_neigbors_labels = [self.y_train[i] for i in k_neigbors_indices]

        # Return the most common class label 
        most_common = Counter(k_neigbors_labels).most_common(1)
        return most_common[0][0]
    
    def predict(self, X_test):
        '''Makes predictions for a set of test data'''
        
        # Loop through each test point and predict its label
        preds = [self.predict_single(x_test_point) for x_test_point in X_test]
        return np.array(preds)
        

In [125]:
# Loading and preparing the data
iris_data = load_iris()
X, y = iris_data.data, iris_data.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

scaler = StandardScaler()
X_train_scld = scaler.fit_transform(X_train)
X_test_scld = scaler.transform(X_test)


In [126]:
# Use the KNN classifier
knn_classifier = KNeighborsClassifier(k=5)

knn_classifier.fit(X_train_scld, y_train)

# Make predictions on the test set
y_predictions = knn_classifier.predict(X_test_scld)

In [127]:
# Evaluate the model
acc = accuracy_score(y_test, y_predictions)

print(f"KNN model accuracy: {acc:.4f}\n")

print("Classification Report")
print(classification_report(y_test, y_predictions, target_names=iris_data.target_names))

KNN model accuracy: 0.9667

Classification Report
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         8
  versicolor       1.00      0.91      0.95        11
   virginica       0.92      1.00      0.96        11

    accuracy                           0.97        30
   macro avg       0.97      0.97      0.97        30
weighted avg       0.97      0.97      0.97        30



## Decision Trees Implementation

In [128]:
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

    def is_leaf_node(self):
        return self.value is not None

In [129]:
class DecisionTree():
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(X.shape[1], self.n_features)
        self.root = self._grow_tree(X, y)

    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))
        
        # Check the stopping criteria
        if (depth >= self.max_depth or n_labels == 1 or n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_features, replace=False)

        # Find the best split
        best_feature, best_threshold = self._best_split(X, y, feat_idxs)

        # Create child nodes 
        left_idxs, right_idxs = self._split(X[:, best_feature], best_threshold)
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feature, best_threshold, left, right)

    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_threshold = None, None

        for feat_idx in feat_idxs:
            X_column = X[:, feat_idx]
            thresholds = np.unique(X_column)

            for thr in thresholds:
                # Calculate the information gain
                gain = self._information_gain(y, X_column, thr)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_threshold = thr

        return split_idx, split_threshold

    def _information_gain(self, y, X_column, threshold):
        # Parent entropy
        parent_entropy = self._entropy(y)

        # Create children
        left_idxs, right_idxs = self._split(X_column, threshold)

        if len(left_idxs) == 0 or len(right_idxs) == 0:
            return 0

        # Calculate the weighted average entropy of children
        n = len(y)
        n_left, n_right = len(left_idxs), len(right_idxs)
        entropy_left, entropy_right = self._entropy(y[left_idxs]), self._entropy(y[right_idxs])
        child_entropy = (n_left / n) * entropy_left + (n_right / n) * entropy_right

        # Calculate the information gain
        information_gain = parent_entropy - child_entropy
        return information_gain

    def _split(self, X_column, split_threshold):
        left_idxs = np.argwhere(X_column <= split_threshold).flatten()
        right_idxs = np.argwhere(X_column > split_threshold).flatten()
        return left_idxs, right_idxs

    def _entropy(self, y):
        # This is similar to a freq vect
        hist = np.bincount(y)
        ps = hist / len(y)

        return -np.sum([p * np.log(p) for p in ps if p > 0])

    def _most_common_label(self, y):
        counter = Counter(y)
        value = counter.most_common(1)[0][0]
        return value

    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])
    
    def _traverse_tree(self, x, node):
        if node.is_leaf_node():
            return node.value
        
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)

In [130]:
# Use decision tree classifier
decision_tree_classifier = DecisionTree(max_depth=3)

decision_tree_classifier.fit(X_train_scld, y_train)

# Make predictions on the test set
dtree_y_predictions = decision_tree_classifier.predict(X_test_scld)

In [131]:
# Evaluate the model
dtree_accuracy = accuracy_score(y_test, dtree_y_predictions)

print(f"Decision Tree model accuracy: {acc:.4f}\n")

print("Classification Report")
print(classification_report(y_test, dtree_y_predictions, target_names=iris_data.target_names))

Decision Tree model accuracy: 0.9667

Classification Report
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00         8
  versicolor       1.00      0.82      0.90        11
   virginica       0.85      1.00      0.92        11

    accuracy                           0.93        30
   macro avg       0.95      0.94      0.94        30
weighted avg       0.94      0.93      0.93        30

